## Excel形式のデータを読み込んでエンコーディング

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

def class_mapping(row):
    # クラス名を数値にマッピング
    mappings = {'障害': 0, 'G1': 10, 'G2': 9, 'G3': 8, '(L)': 7, 'オープン': 7, 'OP': 7, '3勝': 6, '1600': 6, '2勝': 5, '1000': 5, '1勝': 4, '500': 4, '新馬': 3, '未勝利': 1}
    for key, value in mappings.items():
        if key in row:
            return value
    return 0

def standardize_times(df, col_name):
    # 走破時間を秒に変換
    time_parts = df[col_name].str.split(':', expand=True)
    seconds = time_parts[0].astype(float) * 60 + time_parts[1].str.split('.', expand=True)[0].astype(float) + time_parts[1].str.split('.', expand=True)[1].astype(float) / 10
    seconds = seconds.ffill()
    
    # 平均と標準偏差を計算し標準化
    mean_seconds = seconds.mean()
    std_seconds = seconds.std()
    df[col_name] = -((seconds - mean_seconds) / std_seconds)
    
    # 外れ値処理
    df[col_name] = df[col_name].apply(lambda x: -3 if x < -3 else (2 if x > 2.5 else x))
    
    # 再度標準化
    mean_seconds_2 = df[col_name].mean()
    std_seconds_2 = df[col_name].std()
    df[col_name] = (df[col_name] - mean_seconds_2) / std_seconds_2
    
    return mean_seconds, std_seconds, mean_seconds_2, std_seconds_2

def add_seasonal_features(df, date_columns):
    # 日付カラムから季節特徴量を追加
    for date_col in date_columns:
        if not np.issubdtype(df[date_col].dtype, np.datetime64):
            df[date_col] = pd.to_datetime(df[date_col])
        df[f'{date_col}_sin'] = np.sin((df[date_col].dt.month - 1) * (2 * np.pi / 12))
        df[f'{date_col}_cos'] = np.cos((df[date_col].dt.month - 1) * (2 * np.pi / 12))

In [8]:
# 設定
yearStart = 2022
yearEnd = 2023
yearList = np.arange(yearStart, yearEnd + 1)
dfs = []

print("ファイル取得：開始")

# Excel形式のファイルを読み込む
for year in yearList:
    file_path = f"data/{year}.xlsx"  # .csv → .xlsx に変更
    if os.path.exists(file_path):
        print(f"読み込み中: {file_path}")
        df = pd.read_excel(file_path, header=0)  # read_excel を使用
        
        # 日付の処理
        if '日付' in df.columns:
            # 日付が既にdatetime型でない場合のみ変換
            if not pd.api.types.is_datetime64_any_dtype(df['日付']):
                # いくつかの日付フォーマットを試す
                for date_format in ['%Y年%m月%d日', '%Y/%m/%d', '%Y-%m-%d']:
                    try:
                        df['日付'] = pd.to_datetime(df['日付'], format=date_format)
                        break
                    except:
                        continue
                else:
                    # どのフォーマットでも失敗した場合は自動推定
                    df['日付'] = pd.to_datetime(df['日付'], errors='coerce')
        
        # 着順の処理
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df = df.dropna(subset=['着順'])
        df['着順'] = df['着順'].astype(int)
        
        # 賞金の処理
        if '賞金' in df.columns:
            # カンマを除去してから数値に変換
            df['賞金'] = df['賞金'].astype(str).str.replace(',', '')
            df['賞金'] = pd.to_numeric(df['賞金'], errors='coerce').fillna(0)
        
        dfs.append(df)
        print(f"  → {len(df)}行のデータを読み込みました")
    else:
        print(f"警告: {file_path} が見つかりません")

if not dfs:
    raise ValueError("データファイルが見つかりません")

df_combined = pd.concat(dfs, ignore_index=True)
print(f"\n合計 {len(df_combined)} 行のデータを読み込みました")
print("ファイル取得：完了")

ファイル取得：開始
読み込み中: data/2022.xlsx
  → 37427行のデータを読み込みました
読み込み中: data/2023.xlsx
  → 37942行のデータを読み込みました

合計 75369 行のデータを読み込みました
ファイル取得：完了


In [9]:
print("データ変換：開始")

# NaNが含まれる行を削除
df_combined = df_combined.dropna(subset=['走破時間'])

# 走破時間の標準化
mean_seconds, std_seconds, mean_seconds_2, std_seconds_2 = standardize_times(df_combined, '走破時間')

print('1回目平均: ' + str(mean_seconds))
print('2回目平均: ' + str(mean_seconds_2))
print('1回目標準偏差: ' + str(std_seconds))
print('2回目標準偏差: ' + str(std_seconds_2))

# 標準化情報をExcelに保存
os.makedirs('config', exist_ok=True)
time_df = pd.DataFrame({
    'Mean': [mean_seconds, mean_seconds_2],
    'Standard Deviation': [std_seconds, std_seconds_2]
}, index=['First Time', 'Second Time'])
time_df.to_excel('config/standard_deviation.xlsx')

# 通過順の平均を計算
if '通過順' in df_combined.columns:
    pas = df_combined['通過順'].str.split('-', expand=True)
    df_combined['通過順'] = pas.astype(float).mean(axis=1)

# マッピング情報を適用
mappings = {
    '性': {'牡': 0, '牝': 1, 'セ': 2},
    '芝・ダート': {'芝': 0, 'ダ': 1, '障': 2},
    '回り': {'右': 0, '左': 1, '芝': 2, '直': 2},
    '馬場': {'良': 0, '稍': 1, '重': 2, '不': 3},
    '天気': {'晴': 0, '曇': 1, '小': 2, '雨': 3, '雪': 4}
}
for column, mapping in mappings.items():
    if column in df_combined.columns:
        df_combined[column] = df_combined[column].map(mapping)

# クラス変換を適用
if 'クラス' in df_combined.columns:
    df_combined['クラス'] = df_combined['クラス'].apply(class_mapping)

print("データ変換：完了")

データ変換：開始
1回目平均: 102.60497154002307
2回目平均: 0.015193978769472035
1回目標準偏差: 29.22003906359807
2回目標準偏差: 0.9422848306121085
データ変換：完了


In [10]:
print("近5走取得：開始")

# データをソート
df_combined.sort_values(by=['馬', '日付'], ascending=[True, False], inplace=True)

features = ['馬番', '騎手', '斤量', 'オッズ', '体重', '体重変化', '上がり', '通過順', '着順', '距離', 'クラス', '走破時間', '芝・ダート', '天気', '馬場']

# 各馬の過去5走の情報をシフトしながら取得し、ffillで欠損値を補完
shifts = {}
for i in range(1, 6):
    shifts[f'日付{i}'] = df_combined.groupby('馬')['日付'].shift(-i)
    for feature in features:
        if feature in df_combined.columns:
            shifts[f'{feature}{i}'] = df_combined.groupby('馬')[feature].shift(-i).ffill()

# 新しい列を一度にDataFrameに追加
df_combined = pd.concat([df_combined, pd.DataFrame(shifts)], axis=1)

# race_idと馬でグルーピングし、最新の特徴量を取得
df_combined = df_combined.groupby(['race_id', '馬'], as_index=False).last()
df_combined.sort_values(by='race_id', ascending=False, inplace=True)

print("近5走取得：終了")

近5走取得：開始
近5走取得：終了


In [11]:
print("日付変換と特徴量エンジニアリング：開始")

df_combined.replace('---', np.nan, inplace=True)

# 距離差と日付差を計算
if '距離' in df_combined.columns and '距離1' in df_combined.columns:
    df_combined['距離差'] = df_combined['距離'] - df_combined['距離1']
if '日付' in df_combined.columns and '日付1' in df_combined.columns:
    df_combined['日付差'] = (df_combined['日付'] - df_combined['日付1']).dt.days

for i in range(1, 5):
    if f'距離{i}' in df_combined.columns and f'距離{i+1}' in df_combined.columns:
        df_combined[f'距離差{i}'] = df_combined[f'距離{i}'] - df_combined[f'距離{i+1}']
    if f'日付{i}' in df_combined.columns and f'日付{i+1}' in df_combined.columns:
        df_combined[f'日付差{i}'] = (df_combined[f'日付{i}'] - df_combined[f'日付{i+1}']).dt.days

# 斤量関連の列を数値に変換
kinryo_columns = ['斤量', '斤量1', '斤量2', '斤量3', '斤量4', '斤量5']
existing_kinryo_cols = [col for col in kinryo_columns if col in df_combined.columns]
df_combined[existing_kinryo_cols] = df_combined[existing_kinryo_cols].apply(pd.to_numeric, errors='coerce')
df_combined['平均斤量'] = df_combined[existing_kinryo_cols].mean(axis=1)

# 騎手の勝率を計算
if '騎手' in df_combined.columns and '着順' in df_combined.columns:
    jockey_win_rate = df_combined.groupby('騎手')['着順'].apply(lambda x: (x == 1).sum() / x.count()).reset_index()
    jockey_win_rate.columns = ['騎手', '騎手の勝率']
    os.makedirs('calc_rate', exist_ok=True)
    jockey_win_rate.to_excel('calc_rate/jockey_win_rate.xlsx', index=False)
    df_combined = pd.merge(df_combined, jockey_win_rate, on='騎手', how='left')

# 各レースの出走頭数を計算
if 'race_id' in df_combined.columns:
    df_combined['出走頭数'] = df_combined.groupby('race_id')['race_id'].transform('count')

# 各馬に対して過去5レースの出走頭数を特徴量として追加
for i in range(1, 6):
    df_combined[f'出走頭数{i}'] = df_combined.groupby('馬')['出走頭数'].shift(i).fillna(0)

# 距離と走破時間からスピードを計算
speed_cols = []
for i in range(1, 6):
    if f'距離{i}' in df_combined.columns and f'走破時間{i}' in df_combined.columns:
        df_combined[f'スピード{i}'] = df_combined[f'距離{i}'] / df_combined[f'走破時間{i}']
        speed_cols.append(f'スピード{i}')

if speed_cols:
    df_combined['平均スピード'] = df_combined[speed_cols].mean(axis=1, skipna=True)
    df_combined.drop(columns=speed_cols, inplace=True)

# 過去5走の賞金を取得し、賞金合計を計算
if '賞金' in df_combined.columns:
    for i in range(1, 6):
        df_combined[f'賞金{i}'] = df_combined.groupby('馬')['賞金'].shift(i)
    df_combined['過去5走の合計賞金'] = df_combined[[f'賞金{i}' for i in range(1, 6)]].sum(axis=1)
    df_combined.drop(columns=[f'賞金{i}' for i in range(1, 6)] + ['賞金'], inplace=True)

print("日付変換と特徴量エンジニアリング：完了")

日付変換と特徴量エンジニアリング：開始
日付変換と特徴量エンジニアリング：完了


In [12]:
print("最終処理：開始")

df_combined.sort_values(by='race_id', ascending=False, inplace=True)

# 日付カラムから年、月、日を抽出
if '日付' in df_combined.columns:
    df_combined['year'] = df_combined['日付'].dt.year
    df_combined['month'] = df_combined['日付'].dt.month
    df_combined['day'] = df_combined['日付'].dt.day

# 季節特徴量を追加
date_columns = ['日付1', '日付2', '日付3', '日付4', '日付5']
existing_date_cols = [col for col in date_columns if col in df_combined.columns]
if existing_date_cols:
    add_seasonal_features(df_combined, existing_date_cols)

# 日付を数値に変換
date_columns = ['日付', '日付1', '日付2', '日付3', '日付4', '日付5']
for col in date_columns:
    if col in df_combined.columns:
        df_combined['year'] = df_combined[col].dt.year
        df_combined['month'] = df_combined[col].dt.month
        df_combined['day'] = df_combined[col].dt.day
        df_combined[col] = (df_combined['year'] - yearStart) * 365 + df_combined['month'] * 30 + df_combined['day']

if 'year' in df_combined.columns:
    df_combined.drop(['year', 'month', 'day'], axis=1, inplace=True)

# 騎手の乗り替わり特徴量を追加
if '騎手' in df_combined.columns:
    df_combined['騎手の乗り替わり'] = df_combined.groupby('馬')['騎手'].transform(lambda x: (x != x.shift()).astype(int))

# カテゴリカル変数のラベルエンコーディング
categorical_features = ['馬', '騎手', '調教師', 'レース名', '開催', '場名', '騎手1', '騎手2', '騎手3', '騎手4', '騎手5']
for i, feature in enumerate(categorical_features):
    if feature in df_combined.columns:
        print(f"\rProcessing feature {i+1}/{len(categorical_features)}: {feature}", end="")
        le = LabelEncoder()
        df_combined[feature] = le.fit_transform(df_combined[feature].astype(str))

print("\n最終処理：完了")

最終処理：開始
Processing feature 11/11: 騎手5
最終処理：完了


In [13]:
# エンコーディング後のデータを保存
print("ファイル出力：開始")

os.makedirs('encoded', exist_ok=True)
output_path = f'encoded/{yearStart}_{yearEnd}encoded_data.csv'
df_combined.to_csv(output_path, index=False)

print(f"ファイル出力：完了")
print(f"出力ファイル: {output_path}")
print(f"データ件数: {len(df_combined)}行")
print(f"カラム数: {len(df_combined.columns)}列")

ファイル出力：開始
ファイル出力：完了
出力ファイル: encoded/2022_2023encoded_data.csv
データ件数: 75369行
カラム数: 138列
